In [19]:
import pandas as pd
import numpy as np
target='15-05-2024/'

In [20]:
df_rust = pd.read_csv(f'../data/{target}rust-cpu-times.csv')
df_rust['version'] = 'rust'
df_python = pd.read_csv(f'../data/{target}python-cpu-times.csv')
df_python['version'] = 'python'
df_jax = pd.read_csv(f'../data/{target}jax-cpu-times.csv')
df_jax['version'] = 'jax'
df = pd.concat([df_rust, df_python, df_jax])
df.columns

Index(['elapsed_real_time_seconds', 'kernel_cpu_seconds', 'user_cpu_seconds',
       'cpu_percentage', 'max_resident_set_size_kb', 'system_page_size_bytes',
       'major_page_faults', 'minor_page_faults',
       'involuntary_context_switches', 'voluntary_context_switches',
       'filesystem_input_reads', 'filesystem_output_writes',
       'command_name_args', 'version'],
      dtype='object')

# Time and CPU

In [21]:
df.groupby('version')['elapsed_real_time_seconds'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
jax,100.0,40.8804,9.767494,12.66,35.7600,42.490,46.7600,57.52
python,100.0,72.7624,23.000197,9.75,64.2275,72.405,86.9675,129.56
rust,100.0,44.3763,33.754931,6.48,22.2675,34.610,51.9900,157.80


In [42]:
def trim_df(df, low=10, high=90):
    lower = np.percentile(df['elapsed_real_time_seconds'], low)
    upper = np.percentile(df['elapsed_real_time_seconds'], high)
    trimmed_df = df[df.elapsed_real_time_seconds.between(lower, upper)]
    return trimmed_df

In [43]:
df_trimmed = pd.concat([trim_df(df_rust), trim_df(df_python), trim_df(df_jax)])
df_trimmed.groupby('version')['elapsed_real_time_seconds'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
jax,80.0,41.94625,5.777381,30.05,37.0500,42.490,46.1925,51.76
python,80.0,74.24875,14.017375,37.36,66.6500,72.405,84.2375,101.54
rust,80.0,38.46375,19.283346,14.72,22.7725,34.610,46.4000,97.41


In [4]:
df.groupby('version')['user_cpu_seconds'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
jax,100.0,53.4547,13.794520,13.21,46.4900,55.505,62.1025,77.06
python,100.0,140.3757,45.612338,13.88,123.0275,140.655,167.9875,251.32
rust,100.0,84.6857,67.194538,10.44,40.0600,65.665,99.6400,310.11


In [5]:
df.groupby('version')['kernel_cpu_seconds'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
jax,100.0,1.1923,0.186493,0.69,1.10,1.215,1.3150,1.55
python,100.0,0.6600,0.115409,0.41,0.58,0.655,0.7400,1.01
rust,100.0,0.5494,0.204524,0.28,0.41,0.490,0.6225,1.19


In [6]:
df['cpu_percentage'] = df['cpu_percentage'].str[:-1].astype(int)
df.groupby('version')['cpu_percentage'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
jax,100.0,132.15,5.069965,109.0,132.00,133.0,135.00,138.0
python,100.0,192.09,5.929033,147.0,192.00,194.0,195.00,196.0
rust,100.0,187.52,13.085947,81.0,186.75,190.0,193.25,197.0


# Memory

'max_resident_set_size_kb',
'system_page_size_bytes',
'major_page_faults',
'minor_page_faults',

In [7]:
df.groupby('version')['max_resident_set_size_kb'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
jax,100.0,677777.96,2493.399221,670268.0,676157.0,677720.0,679640.0,683188.0
python,100.0,461633.48,3067.518124,454632.0,459460.0,461820.0,463293.0,468736.0
rust,100.0,388399.48,2580.186425,383108.0,386393.0,388460.0,390532.0,393912.0


In [8]:
df.groupby('version')['major_page_faults'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
jax,100.0,15.27,152.599023,0.0,0.0,0.0,0.0,1526.0
python,100.0,17.65,176.500000,0.0,0.0,0.0,0.0,1765.0
rust,100.0,8.51,85.100000,0.0,0.0,0.0,0.0,851.0


In [9]:
df.groupby('version')['minor_page_faults'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
jax,100.0,183657.02,12088.650596,144163.0,178265.75,185241.5,191727.25,215901.0
python,100.0,112795.52,7270.992248,92293.0,108342.75,111448.0,117364.25,138083.0
rust,100.0,100361.61,7476.185288,88563.0,95009.00,98662.0,105037.25,121276.0


# SO

'involuntary_context_switches',
'voluntary_context_switches', 
'filesystem_input_reads',
'filesystem_output_writes',

In [10]:
df.groupby('version')['involuntary_context_switches'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
jax,100.0,2312.84,1210.554191,541.0,1570.5,2035.0,2606.25,8024.0
python,100.0,373.57,67.504471,200.0,350.5,378.5,421.00,524.0
rust,100.0,227.23,91.766816,110.0,160.5,203.5,256.25,550.0


In [11]:
df.groupby('version')['voluntary_context_switches'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
jax,100.0,131012.13,36408.807227,25780.0,112448.0,137465.0,153734.25,197327.0
python,100.0,5682.02,2361.481191,3218.0,3313.5,6007.0,6609.50,14702.0
rust,100.0,3534.72,1509.401026,1937.0,1944.5,3351.5,4246.00,8859.0


In [12]:
df.groupby('version')['filesystem_input_reads'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
jax,100.0,4546.24,45447.856827,0.0,0.0,0.0,0.0,454480.0
python,100.0,6457.04,64570.400000,0.0,0.0,0.0,0.0,645704.0
rust,100.0,2196.24,21962.400000,0.0,0.0,0.0,0.0,219624.0


In [13]:
df.groupby('version')['filesystem_output_writes'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
jax,100.0,8.0,0.0,8.0,8.0,8.0,8.0,8.0
python,100.0,8.0,0.0,8.0,8.0,8.0,8.0,8.0
rust,100.0,8.0,0.0,8.0,8.0,8.0,8.0,8.0


# Rewards

In [14]:
df_rust = pd.read_csv(f'../data/{target}rust-cpu-rewards.csv')
df_python = pd.read_csv(f'../data/{target}python-cpu-rewards.csv')
df_jax = pd.read_csv(f'../data/{target}jax-cpu-rewards.csv')
df = pd.concat([df_rust, df_python, df_jax])
df.columns

Index(['version', 'seed', 'training_steps', 'reward_mean', 'reward_std'], dtype='object')

In [15]:
df.groupby('version')['training_steps'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
jax,100.0,36174.08,10133.772630,6000.0,32000.0,38000.0,42250.0,50176.0
python,100.0,26531.76,8895.125272,2000.0,23000.0,27000.0,32000.0,50176.0
rust,100.0,10400.00,8561.128973,1000.0,4750.0,8000.0,12250.0,39000.0


In [16]:
df.groupby('version')['reward_mean'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
jax,100.0,465.516,102.516670,60.300000,495.975,500.0,500.0,500.0
python,100.0,495.253,19.311406,337.600000,500.000,500.0,500.0,500.0
rust,100.0,498.263,11.527961,402.100006,500.000,500.0,500.0,500.0


In [17]:
df.groupby('version')['reward_std'].describe()

,count,mean,std,min,25%,50%,75%,max
version,,,,,,,,
jax,100.0,10.039423,26.881047,0.0,0.0,0.0,3.52971,157.089147
python,100.0,6.537233,20.557888,0.0,0.0,0.0,0.00000,110.048898
rust,100.0,3.098378,21.180631,0.0,0.0,0.0,0.00000,195.800125
